In [1]:
import tensorflow as tf
import tensorflow.keras.optimizers
import tensorflow.keras.metrics
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
import numpy as np
import pickle
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cf
import pickle 
from datetime import date
import xarray as xr
import netCDF4 as nc
from ndays import numOfDays
import glob
import calendar
import os
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix
import math

2025-02-25 16:57:19.720683: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740502639.731464   21354 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740502639.734551   21354 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-25 16:57:19.746208: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/jawahm/miniforge3/lib/python3.10/site-packages/tensorflow/python/keras/engine/training_arrays_v1.py:37: UserWarning: A 

In [2]:
# Define domain and time period
start_lat = 5  #
end_lat = 25   #
start_lon = -18 #
end_lon = -2  #
start_year = '2006'
end_year = '2019'
start_month = '08' #6
end_month = '10'  #9
start_day = '01'
end_day = '31'
lead_time = 1



In [3]:
# get native MSG grid (core)
coords_filename= '../nxny1640_580_nxnyds164580_blobdx0.04491576_area4_n23_20_32.nc'#[0]  # this is /prj/Africa_cloud/geoloc/*.npz on the Linux system
msg_file = xr.open_dataset(coords_filename).squeeze() # pick any convective core file from ch9_wavelet. 
mlon = msg_file['lons_mid'].values
mlat = msg_file['lats_mid'].values
# find core indices using one file
lat_ind = np.where((mlat[:,1]>=start_lat) & (mlat[:,1]<=end_lat))[0]
lon_ind = np.where((mlon[1,:]>=start_lon) & (mlon[1,:]<=end_lon))[0]
lat = mlat[lat_ind[0]:lat_ind[-1]+1,lon_ind[0]:lon_ind[-1]+1]
lon = mlon[lat_ind[0]:lat_ind[-1]+1,lon_ind[0]:lon_ind[-1]+1]




In [4]:
# read in cores data
list_of_input_file_names = []

for m in range(int(start_month),int(end_month)+1,1): # 1 to include the end month
    if m<10:
        month = str(m).zfill(2)
    else:
        month = str(m)  
    for d in range(1,calendar.monthrange(int(start_year), m)[1]+1,1): # 1 to include the end month
        if d<10:
            day = str(d).zfill(2)
        else:
            day = str(d)  
            #core_filename= '/prj/nflics/hist_cores/2008/06/01/Hist_cores_wa_200806011045.nc'
        dir_name = '/prj/nflics/hist_cores/'+start_year+'/'+month+'/'+day+'/'
        all_file_names=sorted(glob.glob(dir_name+"Hist_cores_wa_*.nc"));
        list_of_input_file_names.append(all_file_names) # all days in month
                  
list_of_input_files=[]                
for a in range(0,len(list_of_input_file_names),1):
    list_of_input_files = list_of_input_files+list_of_input_file_names[a]

list_of_output_files=list_of_input_files[3*4*lead_time:]
list_of_input_files=list_of_input_files[0:-3*4*lead_time]
# for i=0 the predition time will be to-2 (i=0), to-1(i=4), to(i=8) to predict at to+xhr (here x=1 so i=12)

print(len(list_of_input_files))
print(len(list_of_output_files))

5844
5844


In [5]:
a= 8 # 85
b= -63 # 98

image_height= 512 #lat
image_width= 512 #lon
num_channels= 3 #  core at t0-, core at t0-1,  

In [6]:
def load_sequence(i):
   # print(i) 
    # Load 3x4 consecutive input frames
    tir = np.zeros((len(lat[:,1]),len(lon[1,:]),3),dtype=float)

    for offset in range(3):
        with nc.Dataset(list_of_input_files[i + 4*offset]) as ds:
            tir_temp =  ds.variables['msg_Tir'][:]
            tir[:,:,offset] = tir_temp[lat_ind[0]:lat_ind[-1]+1,lon_ind[0]:lon_ind[-1]+1]/10000    

    tir_t_0 = tir[a:,:b,:]
    ind_tir = np.where(tir_t_0>-0.01)
    tir_t_0[ind_tir] = 0
    tir_t_0[np.isnan(tir_t_0)] = 0
    tir_t_0 = np.round(tir_t_0/-173,4)
    x_train = tir_t_0
    
    # Load the output frame
    #target data
    cores = np.zeros((len(lat[:,1]),len(lon[1,:])),dtype=float)
    time_core = np.zeros((1)) 

    #for offset in range(4):
    core_filename = list_of_output_files[i]  
    ds = xr.open_dataset(core_filename).squeeze() 
    core_temp = ds['msg_cores'].values
    cores = core_temp[lat_ind[0]:lat_ind[-1]+1,lon_ind[0]:lon_ind[-1]+1]
    time_core = str(core_filename[-15:-3])

    ind = np.where(cores>0)
    cores[ind] = 1
    y_train= cores[a:,:b]
    y_train=np.expand_dims(y_train, axis=2)
   
    prediction_time = time_core
    time_of_day_tr= np.zeros((image_height, image_width,1))
    time_of_day = float(str(prediction_time)[-6:])/2345
    time_of_day_tr[:,:,:]=round(np.sin(time_of_day*math.pi),2)

    # define input and output tensors
    input_tensor = [x_train,time_of_day_tr]
    output_tensor= y_train

    return input_tensor, output_tensor



In [7]:
def load_sequence_wrapper(i):
    return tf.numpy_function(load_sequence, [i], [tf.float32, tf.float32])



In [8]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input, concatenate, Conv2D, MaxPooling2D, Activation, UpSampling2D)
from tensorflow.keras import layers


# define conv layer
def conv_layer(x_in, filters, gn_num, strides = 1, shape = (image_height,image_width), name=None):
    x = Conv2D(filters, 5, strides=strides, padding='same')(x_in)
    x = Activation('relu', name = name)(x)
   ## x = layers.BatchNormalization()(x)
    #x = tfa.layers.GroupNormalization(groups=gn_num, axis=3)(x) # could replace this with BatchNorm
    return x


# def basic unet structure
def unet_basic(input_shape=(image_height, image_width, num_channels), meta_input_shape=(image_height, image_width,1) ,chan_num=num_channels):

    inputs = Input(shape=input_shape)    # 256 x 256 x 2
    meta_inputs = Input(shape=meta_input_shape)

    # downsample
    down2 = conv_layer(inputs, 4, 1)     # 256 x 256 x 4
    down2_pool = MaxPooling2D((2, 2), strides=None)(down2)   # 128 x 128 x 4

    down3 = conv_layer(down2_pool, 8, None) # 128 x 128 x 8
    down3_pool = MaxPooling2D((4, 4), strides=None)(down3) # 32 x 32 x 8
 
    center = conv_layer(down3_pool, 16, None) # 32 x 32 x 16
    center = conv_layer(center, 8, None) # 32 x 32 x 8
  
    up3 = UpSampling2D((4, 4))(center) # 128 x 128 x 8
    up3 = concatenate([down3, up3], axis=3) # 128 x 128 x 16
    up3 = conv_layer(up3, 4, None) # 128 x 128 x 4

    up2 = UpSampling2D((2, 2))(up3) # 256 x 256 x 4
    up2 = concatenate([down2, up2, meta_inputs], axis=3) # 256 x 256 x 9
    up2 = conv_layer(up2, 4, None) # 256 x 256 x 4
  
    # predict
    output = Conv2D(1, (1, 1))(up2)
    output1 = layers.Activation('sigmoid', dtype='float32', name='prob_pred')(output)
    
    # create model object
    unet = Model(inputs=[inputs,meta_inputs], outputs=output1, name = 'prob_map_unet')
    return unet


unet_model = unet_basic(input_shape=(image_height, image_width, num_channels),meta_input_shape=(image_height, image_width,1), chan_num=num_channels)


I0000 00:00:1740502641.780357   21354 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 43573 MB memory:  -> device: 0, name: NVIDIA A40, pci bus id: 0000:02:00.0, compute capability: 8.6


In [9]:
def _create_mean_filter(half_num_rows, half_num_columns, num_channels):
    """Creates convolutional filter that computes mean.

    M = number of rows in filter
    N = number of columns in filter
    C = number of channels

    :param half_num_rows: Number of rows on either side of center.  This is
        (M - 1) / 2.
    :param half_num_columns: Number of columns on either side of center.  This
        is (N - 1) / 2.
    :param num_channels: Number of channels.
    :return: weight_matrix: M-by-N-by-C-by-C numpy array of filter weights.
    """

    num_rows = 2 * half_num_rows + 1
    num_columns = 2 * half_num_columns + 1
    weight = 1. / (num_rows * num_columns)

    return np.full(
        (num_rows, num_columns, num_channels, num_channels), weight,
        dtype=np.float32
    )

def FSS_loss(target_tensor, prediction_tensor):
    
    half_window_size_px=2
    use_as_loss_function=True 
    #mask_matrix
    function_name=None
    test_mode=False
    """Fractions skill score (FSS).

    M = number of rows in grid
    N = number of columns in grid

    :param half_window_size_px: Number of pixels (grid cells) in half of
        smoothing window (on either side of center).  If this argument is K, the
        window size will be (1 + 2 * K) by (1 + 2 * K).
    :param use_as_loss_function: Boolean flag.  FSS is positively oriented
        (higher is better), but if using it as loss function, we want it to be
        negatively oriented.  Thus, if `use_as_loss_function == True`, will
        return 1 - FSS.  If `use_as_loss_function == False`, will return just
        FSS.
    :param mask_matrix: M-by-N numpy array of Boolean flags.  Grid cells marked
        "False" are masked out and not used to compute the loss.
    :param function_name: Function name (string).
    :param test_mode: Leave this alone.
    :return: loss: Loss function (defined below).
    """

    weight_matrix = _create_mean_filter(
        half_num_rows=half_window_size_px,
        half_num_columns=half_window_size_px, num_channels=1
    )
        
    """Computes loss (fractions skill score).

        :param target_tensor: Tensor of target (actual) values.
        :param prediction_tensor: Tensor of predicted values.
        :return: loss: Fractions skill score.
    """

    smoothed_target_tensor = K.conv2d(
        x=target_tensor, kernel=weight_matrix,
        padding='same', strides=(1, 1), data_format='channels_last'
    )

    smoothed_prediction_tensor = K.conv2d(
        x=prediction_tensor, kernel=weight_matrix,
        padding='same', strides=(1, 1), data_format='channels_last'
    )

    #smoothed_target_tensor = smoothed_target_tensor * eroded_mask_matrix
    #smoothed_prediction_tensor = smoothed_prediction_tensor * eroded_mask_matrix)

    actual_mse = K.mean(
        (smoothed_target_tensor - smoothed_prediction_tensor) ** 2
    )
    reference_mse = K.mean(
        smoothed_target_tensor ** 2 + smoothed_prediction_tensor ** 2
    )

    if use_as_loss_function:
        return actual_mse / reference_mse

    return 1. - actual_mse / reference_mse

    if function_name is not None:
        loss.__name__ = function_name

In [10]:
from tensorflow.python.keras.engine import data_adapter

def _is_distributed_dataset(ds):
    return isinstance(ds, data_adapter.input_lib.DistributedDatasetSpec)

data_adapter._is_distributed_dataset = _is_distributed_dataset

tf.config.run_functions_eagerly(True)

In [11]:
# 2. Build dataset
dataset = tf.data.Dataset.range(len(list_of_input_files) - 3*4) 
#dataset = dataset.shuffle(buffer_size=100)  # shuffle if desired
#dataset = dataset.map(load_sequence, num_parallel_calls=tf.data.AUTOTUNE)
dataset = dataset.map(load_sequence_wrapper, num_parallel_calls=1) #tf.data.AUTOTUNE)
dataset = dataset.batch(8)                 # batch size
#dataset = dataset.prefetch(tf.data.AUTOTUNE)



/home/jawahm/miniforge3/lib/python3.10/site-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


In [12]:
unet_model.compile(optimizer=tensorflow.keras.optimizers.Adam(),
                  loss=FSS_loss,
                  metrics=[tf.keras.metrics.Accuracy()])
#                  metrics=metric_list)

In [13]:
epochs =  20

# Fit the model to the training data.
model_history = unet_model.fit(dataset,epochs=epochs)
#    callbacks=[early_stopping, reduce_lr],


Epoch 1/20


/home/jawahm/miniforge3/lib/python3.10/site-packages/tensorflow/python/util/numpy_compat.py:82: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.asarray(values, dtype=dtype, order=order)
2025-02-25 16:57:22.524459: W tensorflow/core/framework/op_kernel.cc:1829] INVALID_ARGUMENT: ValueError: could not broadcast input array from shape (512,512,3) into shape (512,512)
Traceback (most recent call last):

  File "/home/jawahm/miniforge3/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 277, in __call__
    return [self._convert(x) for x in ret]

  File "/home/jawahm/miniforge3/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 277, in <listcomp>
    return [self._convert(x) for x in ret]

  File "/home/jawahm/miniforge3/lib

(512, 512, 3)
(512, 512, 1)
(512, 512, 1)
(512, 512, 3)
(512, 512, 1)
(512, 512, 1)


2025-02-25 16:57:22.662271: W tensorflow/core/framework/op_kernel.cc:1829] INVALID_ARGUMENT: ValueError: could not broadcast input array from shape (512,512,3) into shape (512,512)
Traceback (most recent call last):

  File "/home/jawahm/miniforge3/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 277, in __call__
    return [self._convert(x) for x in ret]

  File "/home/jawahm/miniforge3/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 277, in <listcomp>
    return [self._convert(x) for x in ret]

  File "/home/jawahm/miniforge3/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 228, in _convert
    result = numpy_compat.np_asarray(value, dtype=dtype, order="C")

  File "/home/jawahm/miniforge3/lib/python3.10/site-packages/tensorflow/python/util/numpy_compat.py", line 82, in np_asarray
    return np.asarray(values, dtype=dtype, order=order)

ValueError: could not broadcast input array from shape (512,512,3) into shape (

InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} ValueError: could not broadcast input array from shape (512,512,3) into shape (512,512)
Traceback (most recent call last):

  File "/home/jawahm/miniforge3/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 277, in __call__
    return [self._convert(x) for x in ret]

  File "/home/jawahm/miniforge3/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 277, in <listcomp>
    return [self._convert(x) for x in ret]

  File "/home/jawahm/miniforge3/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 228, in _convert
    result = numpy_compat.np_asarray(value, dtype=dtype, order="C")

  File "/home/jawahm/miniforge3/lib/python3.10/site-packages/tensorflow/python/util/numpy_compat.py", line 82, in np_asarray
    return np.asarray(values, dtype=dtype, order=order)

ValueError: could not broadcast input 

(512, 512, 3)
(512, 512, 1)
(512, 512, 1)


2025-02-25 16:57:22.896801: W tensorflow/core/framework/op_kernel.cc:1829] INVALID_ARGUMENT: ValueError: could not broadcast input array from shape (512,512,3) into shape (512,512)
Traceback (most recent call last):

  File "/home/jawahm/miniforge3/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 277, in __call__
    return [self._convert(x) for x in ret]

  File "/home/jawahm/miniforge3/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 277, in <listcomp>
    return [self._convert(x) for x in ret]

  File "/home/jawahm/miniforge3/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 228, in _convert
    result = numpy_compat.np_asarray(value, dtype=dtype, order="C")

  File "/home/jawahm/miniforge3/lib/python3.10/site-packages/tensorflow/python/util/numpy_compat.py", line 82, in np_asarray
    return np.asarray(values, dtype=dtype, order=order)

ValueError: could not broadcast input array from shape (512,512,3) into shape (